In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.io as spio
import os
import h5py
from PIL import Image
from pathlib import Path
from tkinter import filedialog
from tkinter import *

#From
def get_file_size(file_name, var_name_hdf5='mov'):
    """ Computes the dimensions of a file or a list of files without loading
    it/them in memory. An exception is thrown if the files have FOVs with
    different sizes
        Args:
            file_name: str or list
                locations of file(s) in memory

            var_name_hdf5: 'str'
                if loading from hdf5 name of the variable to load

        Returns:
            dims: list
                dimensions of FOV

            T: list
                number of timesteps in each file
    """
    if isinstance(file_name, str):
        if os.path.exists(file_name):
            _, extension = os.path.splitext(file_name)[:2]
            extension = extension.lower()
            if extension == '.tif' or extension == '.tiff':
                tffl = tifffile.TiffFile(file_name)
                siz = tffl.series[0].shape
                T, dims = siz[0], siz[1:]
            elif extension == '.avi':
                cap = cv2.VideoCapture(file_name)
                dims = [0, 0]
                try:
                    T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    dims[1] = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    dims[0] = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                except():
                    print('Roll back to opencv 2')
                    T = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))
                    dims[1] = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH))
                    dims[0] = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT))
            elif extension == '.mmap':
                filename = os.path.split(file_name)[-1]
                Yr, dims, T = load_memmap(os.path.join(
                        os.path.split(file_name)[0], filename))
            elif extension in ('.h5', '.hdf5', '.nwb'):
                with h5py.File(file_name, "r") as f:
                    kk = list(f.keys())
                    if len(kk) == 1:
                        siz = f[kk[0]].shape
                    elif var_name_hdf5 in f:
                        if extension == '.nwb':
                            siz = f[var_name_hdf5]['data'].shape
                        else:
                            siz = f[var_name_hdf5].shape
                    else:
                        logging.error('The file does not contain a variable' +
                                      'named {0}'.format(var_name_hdf5))
                        raise Exception('Variable not found. Use one of the above')
                T, dims = siz[0], siz[1:]
            elif extension in ('.sbx'):
                from ...base.movies import loadmat_sbx
                info = loadmat_sbx(file_name[:-4]+ '.mat')['info']
                dims = tuple((info['sz']).astype(int))
                # Defining number of channels/size factor
                if info['channels'] == 1:
                    info['nChan'] = 2
                    factor = 1
                elif info['channels'] == 2:
                    info['nChan'] = 1
                    factor = 2
                elif info['channels'] == 3:
                    info['nChan'] = 1
                    factor = 2
            
                # Determine number of frames in whole file
                T = int(os.path.getsize(
                    file_name[:-4] + '.sbx') / info['recordsPerBuffer'] / info['sz'][1] * factor / 4 - 1)
                
            else:
                raise Exception('Unknown file type')
            dims = tuple(dims)
        else:
            raise Exception('File not found!')
    elif isinstance(file_name, list):
        if len(file_name) == 1:
            dims, T = get_file_size(file_name[0], var_name_hdf5=var_name_hdf5)
        else:
            dims, T = zip(*[get_file_size(fn, var_name_hdf5=var_name_hdf5)
                for fn in file_name])
            if len(list(set(dims))) > 1:
                raise Exception("Files have different FOV sizes.")
            else:
                dims = dims[0]
    else:
        raise Exception('Unknown input type')
    return dims, T








In [ ]:
root = Tk()
root.withdraw()
start_dir = filedialog.askdirectory()
data_filename =  filedialog.askopenfilename(title = "Select data file", filetypes = (("HDF5 files","*.hdf5"),))

#Load in the respective H5 file
f = h5py.File(data_filename, 'r')

# Get the data, C is denoised and deconvolved with raw amplitude, F_dff is the raw signal converted to df/f with some deconvolution
F_dff = f['estimates/F_dff']


What do I need to do?
Keep the GUI because it makes the script more portable and will give people something to do while I do higher analysis?
Separate AM/PM into subfolders, each folder has the HDF5, the sub-videos, Putty-log
Have user choose directory and input movies to analyze (does the n first)
Load in movies to get frames
Load in the 

In [ ]:
#Scanbox event frame data 
mat_filename = filedialog.askopenfilename(title = "Select event file", filetypes = (("Matlab files","*.mat"),), multiple=True)
mat_filename = sorted(mat_filename)
count = 0
end_frame = 0
for file in mat_filename:
    #First check if respective tiff file exists
    tiff_name = Path(file).stem
    config = Path(tiff_name + '.tif')
    if config.is_file():
        # Load configuration file values
        if count = 0:
            mat_data = spio.loadmat(file, struct_as_record=False)
            events = mat_data['info'][0,0].frame
            end_frame = 
            count = count + 1
        else:
            mat_data = spio.loadmat(file, struct_as_record=False)
            events = np.concatenate([events, mat_data['info'][0,0].frame])
            count = count + 1
            
def get_frames(tiff_input):
    """This function takes a path to a Tiff as input and returns the frame length"""
    
    
    
    
    
    
    


In [ ]:
##Code to break apart movie by trials

#Define global start point and counter
start = 0
counter = 0

#Cycle through trials and transform 2D into 3D matrix for easy analysis
for trial_len in frames:
    #Select all cells, and subset of time for respective trial
    #Then put that matrix into a slice of 3D matrix
    trials[:,0:trial_len,counter] = F_dff[:,start:(start+trial_len)]
    start = start + trial_len
    counter = counter + 1
    

    
#Average cells across trials (across third dimension)
avg_trial = trials.mean(2)

In [ ]:
#Lick and event timing data in milliseconds from start
lick_filename =  filedialog.askopenfilename(title = "Select arduino output file", filetypes = (("txt files","*.txt"),))
lickdf = pd.read_csv(lick_filename, sep="\t")

In [ ]:
#Get path to tifs, open tifs, count frames, store in list 
tiffs_path = "/home/tomi/Desktop/NaomiVideos/2D1/Day1/"

frames = [] #Declare an empty list
files  = sorted(os.listdir(tiffs_path)) #Make sure files are loaded in order of naming convention
        
for filename in files:
    if filename.endswith(".tif"):
        img = Image.open(tiffs_path + filename) #Define absolute path to file by concatenating the two strings
        #print(filename)
        frames.append(img.n_frames)

#Preallocating a 3D matrix for data handling based on dimensions of data and frames; cells x frames x trials
trials = np.zeros((len(F_dff),max(frames),len(frames)))

In [ ]:
#After data load we need cut up data relative to events
#I can use odd and even as a logic source as the first event is always air and the second water
# Every odd frames event will correspond to air event or just stack air and water events in order and mesh with frames
# Use to find time to frame conversion

eventsdf = lickdf[(lickdf['Event'] == 'Air') | (lickdf['Event'] == 'Water')]
print(eventsdf, events)

In [ ]:
#All cells, all trials
fig, ax =  plt.subplots(figsize=(20,5))
im = ax.plot(F_dff[:].T)

fig.tight_layout()
#plt.savefig('hal3_102819_2.svg')
plt.show()

In [ ]:
#All cells, avg trial
fig, ax =  plt.subplots(figsize=(10,10))
im = ax.imshow(avg_trial)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Cell number')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average cell activity across trials, 2D1 day 1')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1-avgcells.png')
plt.show()
    

In [ ]:
#Single cell, all trials
for cell_num in range(len(F_dff)):
    s_cell = trials[cell_num,:,:].T

    fig, ax =  plt.subplots(figsize=(6,4))
    im = ax.imshow(s_cell)

    cbar = ax.figure.colorbar(im, ax=ax)

    plt.ylabel('Trials')
    plt.xlabel('Frames (15.49 Hz)')
    plt.title('Single cell activity across trials, Cell #:{0} Day 1'.format(cell_num))
    cbar.ax.set_ylabel('df/f')
    fig.tight_layout()
    sname = '2D1_day1-{0}.png'.format(cell_num) 
    plt.savefig(sname)
    plt.show()
    


In [ ]:
#Plot average cell activity of just D1 cells
d1_list = [12,24,25,29,32,34,36,39]
d1_avg = np.zeros((len(d1_list),max(frames)))

counter = 0

for cell in d1_list:
    d1_avg[counter,:] = avg_trial[cell,:]
    counter = counter + 1

#D1 cells, avg trial
fig, ax =  plt.subplots(figsize=(10,2))
im = ax.imshow(d1_avg)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Cell number')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average D1 cell activity across trials, 2D1 day 1 (Example)')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1example-avgD1cells.png')
plt.show()    






In [ ]:
#Plot group average activity of D1 cells
d1_grp = np.zeros((len(d1_list),max(frames),len(frames)))

counter = 0

for cell in d1_list:
    d1_grp[counter,:,:] = trials[cell,:,:]
    counter = counter + 1

d1_grp_avg = d1_grp.mean(0)

    
#Day1 active cells, avg trial
fig, ax =  plt.subplots(figsize=(9,3))
im = ax.imshow(d1_grp_avg.T)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Trial')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average D1 group activity across trials, 2D1 day 1')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1-avgd1group.png')
plt.show()    

In [ ]:
#Plot average cell activity of just D2 cells
d2_list = [3,5,8,9,11,18,19,20,22,26,28,30,31,37,38]
d2_avg = np.zeros((len(d2_list),max(frames)))

counter = 0

for cell in d2_list:
    d2_avg[counter,:] = avg_trial[cell,:]
    counter = counter + 1

#D2 cells, avg trial
fig, ax =  plt.subplots(figsize=(10,2))
im = ax.imshow(d2_avg)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Cell number')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average D2 cell activity across trials, 2D1 day 1 (Example)')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1example-avgD2cells.png')
plt.show()    



In [ ]:
#Plot group average activity of D2 cells
d2_grp = np.zeros((len(d2_list),max(frames),len(frames)))

counter = 0

for cell in d2_list:
    d2_grp[counter,:,:] = trials[cell,:,:]
    counter = counter + 1

d2_grp_avg = d2_grp.mean(0)

    
#Day1 active cells, avg trial
fig, ax =  plt.subplots(figsize=(9,3))
im = ax.imshow(d2_grp_avg.T)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Trial')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average D2 group activity across trials, 2D1 day 1')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1-avgd2group.png')
plt.show()    

In [ ]:
#Plot average cell activity of active cells for classification
day1_list = [106,107,108,109,110,111,112,118,119,120,121,122,123,124,133,134,135,136,137,139,140,141,142,149,160,161,187,188,189,190,191,192,193,194,195,196,197,199,203,205,206,208,209,210,212,214,219,220,225,231,232,233,237,239,240]
day1_avg = np.zeros((len(day1_list),max(frames)))

counter = 0

for cell in day1_list:
    day1_avg[counter,:] = avg_trial[cell,:]
    counter = counter + 1

#Day1 active cells, avg trial
fig, ax =  plt.subplots(figsize=(15,5))
im = ax.imshow(day1_avg)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Cell number')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average active cell activity across trials, 2D1 day 1')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1-avggoodcells.png')
plt.show()    


In [ ]:
#Single cell, all trials for active cells
for cell_num in day1_list:
    s_cell = trials[cell_num,:,:].T

    fig, ax =  plt.subplots(figsize=(6,3))
    im = ax.imshow(s_cell)

    cbar = ax.figure.colorbar(im, ax=ax)

    plt.ylabel('Trials')
    plt.xlabel('Frames (15.49 Hz)')
    plt.title('Single cell activity across trials, Cell #:{0} Day 1 (Active)'.format(cell_num))
    cbar.ax.set_ylabel('df/f')
    fig.tight_layout()
    sname = '2D1_day1-{0}_active.png'.format(cell_num) 
    plt.savefig(sname)
    plt.show()

In [ ]:
#Plot average cell activity of active Up cells for classification
up_list = [106,109,110,112,119,120,121,122,123,135,140,141,142,149,161,187,193,195,199,205,206,210]
up_avg = np.zeros((len(up_list),max(frames)))

counter = 0

for cell in up_list:
    up_avg[counter,:] = avg_trial[cell,:]
    counter = counter + 1

#Day1 active cells, avg trial
fig, ax =  plt.subplots(figsize=(15,3))
im = ax.imshow(up_avg)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Cell number')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average up cell activity across trials, 2D1 day 1')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1-avgupcells.png')
plt.show()    


In [ ]:
#Plot average cell activity of active Down cells for classification
dwn_list = [107,108,118,124,133,134,136,137,188,189,190,196,197,208,209,212,231,239,240]
dwn_avg = np.zeros((len(dwn_list),max(frames)))

counter = 0

for cell in dwn_list:
    dwn_avg[counter,:] = avg_trial[cell,:]
    counter = counter + 1

#Day1 active cells, avg trial
fig, ax =  plt.subplots(figsize=(15,2))
im = ax.imshow(dwn_avg)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Cell number')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average down cell activity across trials, 2D1 day 1')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1-avgdowncells.png')
plt.show()    


In [ ]:
#Plot group average activity of active Up cells for classification
up_list = [106,109,110,112,119,120,121,122,123,135,140,141,142,149,161,187,193,195,199,205,206,210]
up_grp = np.zeros((len(up_list),max(frames),len(frames)))

counter = 0

for cell in up_list:
    up_grp[counter,:,:] = trials[cell,:,:]
    counter = counter + 1

up_grp_avg = up_grp.mean(0)

    
#Day1 active cells, avg trial
fig, ax =  plt.subplots(figsize=(8,5))
im = ax.imshow(up_grp_avg.T)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Trial')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average up group activity across trials, 2D1 day 1')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1-avgupgroup.png')
plt.show()    

In [ ]:
#Plot group average activity of active Down cells for classification
dwn_list = [107,108,118,124,133,134,136,137,188,189,190,196,197,208,209,212,231,239,240]
dwn_grp = np.zeros((len(dwn_list),max(frames),len(frames)))

counter = 0

for cell in dwn_list:
    dwn_grp[counter,:,:] = trials[cell,:,:]
    counter = counter + 1

dwn_grp_avg = dwn_grp.mean(0)

    
#Day1 active cells, avg trial
fig, ax =  plt.subplots(figsize=(8,5))
im = ax.imshow(dwn_grp_avg.T)

cbar = ax.figure.colorbar(im, ax=ax)

plt.ylabel('Trial')
plt.xlabel('Frames (15.49 Hz)')
plt.title('Average down group activity across trials, 2D1 day 1')
cbar.ax.set_ylabel('df/f')
fig.tight_layout()
plt.savefig('2D1_day1-avgdowngroup.png')
plt.show()    